In [8]:
from imports import *
from funcs import *

In [19]:
climate_var = "climate-data\A2\HADCM3_A2c_WIND_2020.dif"

global temps_mat

temps_mat = conv(climate_var)[0]
temps_df = conv(climate_var)[1]

month_dict = dict(zip(month, range(0,12)))

fig = plt.figure()
plt.rcParams['figure.figsize'] = [10, 20]

def plotter(month):
    ax = plt.axes(projection=ccrs.PlateCarree())

    data, lons = add_cyclic_point(temps_mat[month_dict[month]], coord = longitudes)

    if 'WIND' in climate_var:
        cs = ax.contourf(lons, latwind, data, levels = np.arange(np.min(temps_mat), np.max(temps_mat), 0.25),
            transform = ccrs.PlateCarree(),cmap='Reds',extend='both')
    
    else:
        cs = ax.contourf(lons, latitudes, data, levels = np.arange(-6, 14, 1.5),
            transform = ccrs.PlateCarree(),cmap='Reds',extend='both')
        

    cb = plt.colorbar(cs, ticks = cs.levels, orientation = 'horizontal', location = 'top', 
                      shrink = 0.75, aspect = 50, pad = 0.01)
    cb.set_label('Temperature Difference [°C]', fontsize = 8)
    cb.ax.tick_params(labelsize=8)

    ax.coastlines()

    ax.set_xticks(np.arange(-180,181,30), crs=ccrs.PlateCarree())
    lon_formatter = cticker.LongitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)

    ax.set_yticks(np.arange(-90,91,15), crs=ccrs.PlateCarree())    
    lat_formatter = cticker.LatitudeFormatter()
    ax.yaxis.set_major_formatter(lat_formatter)


    ax.tick_params(axis='both', which='major', labelsize=8)

    ax.gridlines()
    plt.rcParams['figure.figsize'] = [10, 20]

    return 

month_slider = widgets.SelectionSlider(
    options=month,
    value='Jan',
    description='Month',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

'''month_slider = widgets.IntSlider(
                value=0,
                min=0,
                max=11,
                step=1,
                continuous_update=True,
                readout=True,
                description = 'Month',)'''
    
interact(plotter, month = month_slider)

#
plt.show()


<Figure size 1000x2000 with 0 Axes>

interactive(children=(SelectionSlider(continuous_update=False, description='Month', options=('Jan', 'Feb', 'Ma…

In [36]:
climate_var = "climate-data\A2\HADCM3_A2c_TEMP_2020.dif"

from funcs import *

var = conv(climate_var)
var

(array([[[ 5.22,  5.22,  5.22, ...,  5.22,  5.22,  5.22],
         [ 5.61,  5.43,  5.53, ...,  5.67,  5.78,  5.87],
         [ 6.73,  5.93,  6.06, ...,  6.12,  6.94,  7.59],
         ...,
         [ 0.49,  0.47,  0.48, ...,  0.36,  0.4 ,  0.47],
         [ 0.52,  0.52,  0.51, ...,  0.5 ,  0.55,  0.55],
         [ 0.6 ,  0.6 ,  0.6 , ...,  0.6 ,  0.6 ,  0.6 ]],
 
        [[ 2.55,  2.55,  2.55, ...,  2.55,  2.55,  2.55],
         [ 2.  ,  1.93,  2.15, ...,  2.13,  2.23,  2.2 ],
         [ 3.8 ,  3.  ,  3.24, ...,  3.28,  4.02,  4.49],
         ...,
         [ 1.11,  1.17,  1.18, ...,  0.75,  0.89,  1.  ],
         [ 1.18,  1.19,  1.18, ...,  1.01,  1.08,  1.15],
         [ 1.06,  1.06,  1.06, ...,  1.06,  1.06,  1.06]],
 
        [[ 3.09,  3.09,  3.09, ...,  3.09,  3.09,  3.09],
         [ 2.99,  3.07,  3.3 , ...,  3.  ,  3.04,  3.02],
         [ 4.28,  4.03,  4.31, ...,  4.27,  4.26,  4.56],
         ...,
         [ 2.01,  1.9 ,  1.76, ...,  2.02,  2.05,  2.07],
         [ 2.07,  2.02, 